In [17]:
import json
import re

study = json.load(open("userstudy.json"))

In [18]:
users = list(study.keys())
users

['2024-3-25-11-8-11',
 '2024-3-25-14-45-41',
 '2024-3-25-20-1-48',
 '2024-3-25-22-10-55',
 '2024-3-26-10-37-31',
 '2024-3-26-12-47-58',
 '2024-3-26-9-56-20']

In [19]:
answers = map(lambda s: list(map(lambda a: a['user'], [ss for ss in s if ss['type'] == 'text'])), study.values())
answers = [x for xs in answers for x in xs] # flatten

In [20]:
def notnum(st):
    return not st.lstrip('-').replace('.','',1).replace('e-','',1).replace('e','',1).isdigit()

def filter_nonnum(answers):
    nonnum = [a for a in answers if notnum(a) ]
    return nonnum

nonnum = filter_nonnum(answers)
nonnum

['>180',
 '29-36',
 '14-43',
 '25-50',
 '<7.1',
 '21-29',
 '14-43',
 '>180',
 '14-43',
 '<7.1',
 '<7.1',
 '-71. -43',
 '7.1-14',
 '14-43',
 '36-43',
 '-100 to -71',
 '-100 to -71',
 '<7.1',
 '-71 to -43',
 '7.1-14',
 '14-21',
 '-100, -71',
 '14-21',
 '<25',
 '25-50',
 '-14--43',
 '21-29',
 '-71--43',
 '21-29',
 '<25',
 '14-43',
 '21-29',
 '<25',
 '<25',
 '14-21',
 '-100--71',
 '25-50',
 '<7.1',
 '40%',
 '<25',
 '-100~-71',
 '<25',
 '14-21',
 '-14~14',
 '21-29',
 '-43~-14',
 '100-130',
 '<25',
 '-100~-71',
 '<25',
 '25-50',
 '25-50',
 '-71~-43',
 '[14, 21]',
 '[0, 25]',
 '>100',
 '[14,21]',
 '<25',
 '[-14, 14]',
 '[7.1,14]',
 '<25',
 '[-100,-71]',
 '[-71,-43]',
 '[21,29]',
 '<25',
 '<25',
 '[29,36]',
 '[-71,-43]',
 '<7.1',
 '<30',
 '>180',
 '[14, 43]',
 '<20',
 '[29, 36]',
 '<70',
 '[-71, -43]',
 '[25, 50]',
 '[29, 36]',
 '[14, 21]',
 '<25',
 '[-100, -71]',
 '[21, 29]',
 '<25',
 '[-14, 14]']

In [21]:
def clean_answer(ans):
    if not notnum(ans):
        return float(ans)
    
    # some exceptions
    ans = ans.replace("-71. -43", "-71--43").replace("<25", "12.5")

    ans = ans.lstrip('<').lstrip('>').lstrip('[').rstrip(']').replace('%', '')

    if 'to' in ans:
        ans = list(map(lambda s: s.strip(' '), ans.split('to')))
    elif ',' in ans:
        ans = list(map(lambda s: s.strip(' '), ans.split(',')))
    elif '~' in ans:
        ans = list(map(lambda s: s.strip(' '), ans.split('~')))
    elif '--' in ans:
        ans = ans.replace('--','@-').split('@')

    if type(ans) is str:
        if ans.count('-') == 2:
            gs = ans.split('-')
            ans = ['-'.join(gs[:2]), '-'.join(gs[2:])]
        elif ans.count('-') == 1 and ans[0] != '-':
            ans = ans.split('-')
    
    if type(ans) is list:
        assert(len(ans) == 2)
        ans = (float(ans[0]) + float(ans[1])) / 2

    return float(ans)

# clean_answer('-71. -43')
list(zip(answers, list(map(lambda c: str(clean_answer(c)), answers))))
# "\n".join(map(lambda t: ",".join(t), list(zip(answers, list(map(clean_answer, answers))))))

[('>180', '180.0'),
 ('0', '0.0'),
 ('29-36', '32.5'),
 ('12.5', '12.5'),
 ('14-43', '28.5'),
 ('90', '90.0'),
 ('90', '90.0'),
 ('-100', '-100.0'),
 ('20', '20.0'),
 ('25-50', '37.5'),
 ('<7.1', '7.1'),
 ('30', '30.0'),
 ('21-29', '25.0'),
 ('12', '12.0'),
 ('14-43', '28.5'),
 ('-60', '-60.0'),
 ('>180', '180.0'),
 ('0', '0.0'),
 ('0', '0.0'),
 ('0', '0.0'),
 ('14-43', '28.5'),
 ('-95', '-95.0'),
 ('17', '17.0'),
 ('<7.1', '7.1'),
 ('<7.1', '7.1'),
 ('13', '13.0'),
 ('0', '0.0'),
 ('10', '10.0'),
 ('-80', '-80.0'),
 ('-71. -43', '-57.0'),
 ('0', '0.0'),
 ('0', '0.0'),
 ('7.1-14', '10.55'),
 ('10', '10.0'),
 ('14-43', '28.5'),
 ('20', '20.0'),
 ('0', '0.0'),
 ('1', '1.0'),
 ('36-43', '39.5'),
 ('50', '50.0'),
 ('-100 to -71', '-85.5'),
 ('5', '5.0'),
 ('0', '0.0'),
 ('200', '200.0'),
 ('-100 to -71', '-85.5'),
 ('30', '30.0'),
 ('<7.1', '7.1'),
 ('13', '13.0'),
 ('0', '0.0'),
 ('0', '0.0'),
 ('-71 to -43', '-57.0'),
 ('-25', '-25.0'),
 ('7.1-14', '10.55'),
 ('15', '15.0'),
 ('10', '10.

In [22]:
questions = map(lambda s: list(map(lambda a: a['q'], s)), study.values())
questions = [x for xs in questions for x in xs] # flatten
questions

['What is your best estimate for population density in this tile?',
 'What is your best estimate for population density in this tile?',
 'What is your best estimate for person of color in this tile?',
 'What is your best estimate for person of color in this tile?',
 'What is your best estimate for dem. lead in this tile?',
 'What is your best estimate for dem. lead in this tile?',
 'What is your best estimate for dem. lead in this tile?',
 'What is your best estimate for dem. lead in this tile?',
 'What is your best estimate for population density in this tile?',
 'What is your best estimate for population density in this tile?',
 'What is your best estimate for person of color in this tile?',
 'What is your best estimate for person of color in this tile?',
 'What is your best estimate for person of color in this tile?',
 'What is your best estimate for person of color in this tile?',
 'What is your best estimate for dem. lead in this tile?',
 'What is your best estimate for dem. lead 

In [23]:
def to_var(q):
    if q == 'population density':
        return 'PopSqKm'
    if q == 'person of color':
        return 'PoC'
    if q == 'dem. lead':
        return 'DemLead'
    assert(True)

def clean_question(qs):
    qs = qs.replace('What is your best estimate for ', '').replace(' in this tile?', '').replace('Which tile has ','').replace(' in range ', '').replace(' and ', '').replace(" in ", "")
    if "[" in qs:
        qs = re.split('\[|\]', qs)
        assert(len(qs) == 5)
        qs = [to_var(qs[0]), to_var(qs[2])]
    else:
        qs = to_var(qs)
    return qs
list(map(clean_question, questions))

['PopSqKm',
 'PopSqKm',
 'PoC',
 'PoC',
 'DemLead',
 'DemLead',
 'DemLead',
 'DemLead',
 'PopSqKm',
 'PopSqKm',
 'PoC',
 'PoC',
 'PoC',
 'PoC',
 'DemLead',
 'DemLead',
 'PopSqKm',
 'PopSqKm',
 'PopSqKm',
 'PopSqKm',
 'DemLead',
 'DemLead',
 'PoC',
 'PoC',
 'PoC',
 'PoC',
 'PopSqKm',
 'PopSqKm',
 'DemLead',
 'DemLead',
 'PopSqKm',
 'PopSqKm',
 'PoC',
 'PoC',
 'DemLead',
 'DemLead',
 'PopSqKm',
 'PopSqKm',
 'PoC',
 'PoC',
 'DemLead',
 'DemLead',
 'PopSqKm',
 'PopSqKm',
 'DemLead',
 'DemLead',
 'PoC',
 'PoC',
 'PopSqKm',
 'PopSqKm',
 'DemLead',
 'DemLead',
 'PoC',
 'PoC',
 'PoC',
 'PoC',
 'PopSqKm',
 'PopSqKm',
 'DemLead',
 'DemLead',
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'DemLead'],
 ['PopSqKm', 'PoC'],
 ['PopSqKm', 'PoC'],
 ['PopSqKm', 'PoC'],
 ['PopSqKm', 'PoC'],
 ['PopSqKm', 'PoC'],
 ['Po

In [24]:
for user in study:
    answers = study[user]
    for ans in answers:
        ans['q_clean'] = clean_question(ans['q'])
        ans['user_clean'] = clean_answer(ans['user']) if ans['type'] == 'text' else ans['user']

open("userstudy_clean.json", "w").write(json.dumps(study))

252142